# SQL grouping and summarizing data (exercises)

## Preparation

For this section you need `chinook.db` database file and working `%sql` magic.  
If you don't have it, please go back to the [previous section](connect_to_database.ipynb) and follow the instructions.  
The following code should not produce any errors:

In [1]:
%load_ext sql
%sql sqlite:///chinook.db

Connecting to 'sqlite:///chinook.db'

## Exercise: summaries per group

Using SQL only create a table containing summary info of the `tracks` table grouped by albums (so, by the `AlbumId` column).  
In the result table report columns:
- `AlbumId`
- `TracksNum`: the total count of tracks in the album
- `TotalPrice`: sum of all track `UnitPrice`s
- `MeanTrackSec`: based on `Milliseconds`, the average time duration of tracks expressed in seconds

Order the final table with decreasing `TotalPrice`. Show first 10 rows.    
Additionally, you may `ROUND` the prices to two positions and times to full seconds.

In [10]:
%%sql

SELECT AlbumId, COUNT(*) AS TracksNum, Round(SUM(UnitPrice), 2) AS TotalPrice, ROUND(AVG(Milliseconds)/1000, 0)  AS MeanTrackSec
    FROM tracks  
    GROUP BY AlbumId  
    ORDER BY TotalPrice DESC 
    LIMIT 10

Running query in 'sqlite:///chinook.db'

AlbumId,TracksNum,TotalPrice,MeanTrackSec
141,57,56.43,264.0
229,26,51.74,2718.0
230,25,49.75,2594.0
251,25,49.75,1533.0
231,24,47.76,2637.0
253,24,47.76,2926.0
228,23,45.77,2599.0
250,22,43.78,1302.0
227,19,37.81,2778.0
261,17,33.83,2322.0


## Exercise: summaries per group (with pandas)

Use another strategy to generate the table from the previous exercise.  
Use SQL only to get the complete `tracks` table into a Pandas `DataFrame`.  
Then, perform the same calculations using Pandas `groupby` and `agg` commands.  

Now, imagine that the database is very large and remote, located on multiple servers somewhere in the world:
- What data are transmitted over the database connection in both exercises?
- Where are the calculations performed in both exercises?
- Which approach would scale better?

In [41]:
import pandas as pd 
import sqlalchemy as sa

engine = sa.create_engine("sqlite:///chinook.db")
sql = sa.text("SELECT * FROM tracks")
with engine.connect() as conn:
    arr = conn.execute(sql).fetchall()
    
    sql = sa.text("SELECT * FROM tracks")
df = pd.read_sql(sql, con=engine)

summary_df = (
    df.groupby("AlbumId")
    .agg(
        TracksNum=("AlbumId", "count"),
        TotalPrice=("UnitPrice", lambda x: round(x.sum(), 2)),
        MeanTrackSec=("Milliseconds", lambda x: round(x.mean() / 1000, 0))
    )
    .sort_values("TotalPrice", ascending=False)
    .head(10)
)

summary_df

,TracksNum,TotalPrice,MeanTrackSec
AlbumId,,,
141,57,56.43,264.0
229,26,51.74,2718.0
230,25,49.75,2594.0
251,25,49.75,1533.0
253,24,47.76,2926.0
231,24,47.76,2637.0
228,23,45.77,2599.0
250,22,43.78,1302.0
227,19,37.81,2778.0


## Exercise: filtering based on summary result

Find `AlbumId`s that have the number of tracks between 18 and 20. Show 5 rows.

In [50]:
%%sql

SELECT AlbumId, COUNT(*) AS TracksNum, Round(SUM(UnitPrice), 2) AS TotalPrice, ROUND(AVG(Milliseconds)/1000, 0)  AS MeanTrackSec
    FROM tracks  
    GROUP BY AlbumId 
    HAVING TracksNum BETWEEN 18 AND 20  
    LIMIT 5

Running query in 'sqlite:///chinook.db'

AlbumId,TracksNum,TotalPrice,MeanTrackSec
21,18,17.82,212.0
37,20,19.8,229.0
54,20,19.8,204.0
55,20,19.8,225.0
72,18,17.82,243.0


## Exercise: concatenating texts in a group

Based on `tracks` grouped by `AlbumId`, create a table with two columns:
- `AlbumId`
- `Tracks`: a semicolon-separated concatenated track names of the album

Show some 5 rows.

In [51]:
%%sql

SELECT AlbumId, GROUP_CONCAT( Name, ";" ) AS Tracks 
  FROM tracks 
  GROUP BY AlbumId
  LIMIT 5

Running query in 'sqlite:///chinook.db'

AlbumId,Tracks
1,For Those About To Rock (We Salute You);Put The Finger On You;Let's Get It Up;Inject The Venom;Snowballed;Evil Walks;C.O.D.;Breaking The Rules;Night Of The Long Knives;Spellbound
2,Balls to the Wall
3,Fast As a Shark;Restless and Wild;Princess of the Dawn
4,Go Down;Dog Eat Dog;Let There Be Rock;Bad Boy Boogie;Problem Child;Overdose;Hell Ain't A Bad Place To Be;Whole Lotta Rosie
5,Walk On Water;Love In An Elevator;Rag Doll;What It Takes;Dude (Looks Like A Lady);Janie's Got A Gun;Cryin';Amazing;Blind Man;Deuces Are Wild;The Other Side;Crazy;Eat The Rich;Angel;Livin' On The Edge
